# Ray RLlib Multi-Armed Bandits - Exploration-Exploitation Strategies

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

What strategy should we follow for selecting actions that balance the exploration-exploitation tradeoff, yielding the maximum average reward over time? This is the core challenge of RL/MAB algorithms.

This lesson has two goals, to give you an intuitive sense of what makes a good algorithm and to introduce several popular examples. 

> **Tip:** For the first time through this material, you may wish to focus on the first goal, developing an intuitive sense of the requirements for a good algorithm. Come back later to explore the details the algorithms discussed.

At least read through the first sections, stopping at _UCB in More Detail_ under _Upper Confidence Bound_. Then proceed to the [next lesson](03-Simple-Multi-Armed-Bandit.ipynb). 

## What Makes a Good Exploration-Exploitation Algorithm?

Let's first assume we are considered only stationary bandits. The ideal algorithm achieves these properties:

1. It explores all the actions reasonably aggressively.
2. When exploring, it picks the action most likely to produce an optimal reward.
3. It converges quickly to the action that optimizes the mean reward.
4. It stops exploration once the optimal action is known and just exploits!

For non-stationary and context bandits, the optimal action may change over time, so some exploration may always be needed. 

## Popular Algorithms

With these properties in mind, let's briefly discuss four algorithms. We'll use two of them in examples over several subsequent lessons.

### $\epsilon$-Greedy

One possible strategy is quite simple, called $\epsilon$-Greedy, where $\epsilon$ is small number that determines how frequently exploration is done. The best-known action is exploited most of the time ("greedily"), governed by probability $1 - \epsilon$ (i.e., in percentage terms $100*(1 - \epsilon)$%). With probability $\epsilon$, a previously-unexplored action is randomly picked in the hopes of finding a new, best-rewarding action.

Typical values of $\epsilon$ are between 0.01 and 0.1. A larger value, like 0.1, explores more aggressively and finds the optimal policy more quickly, but afterwards the aggressive exploration strategy becomes a liability, as it only selects the optimal action ~90% of the time, continuing excessive exploration that is now counterproductive. In contrast, smaller values, like 0.01, are slower to find the optimal policy, but once found continue to select it ~99% of the time, so over time the mean reward is _higher_ for _smaller_ $\epsilon$ values, as the optimal action is selected more often.

How does it stack up against our properties:

1. The higher the $\epsilon$ value, the more quickly the action space is explored.
2. It randomly picks the next action, so there is no "intelligence" involved in optimizing the choice.
3. The higher the $\epsilon$ value, the more quickly the optimal action is found.
4. Just as this algorithm makes no attempt to optimize the choice of action during exploration, it makes no attempt to throttle back exploration when the optimal value is found. 

To address point 4, you could adopt an enhancement that decays the $\epsilon$ value over time, rather than keeping it fixed.

See [Wikipedia - MAB Approximate Solutions](https://en.wikipedia.org/wiki/Multi-armed_bandit) and [Sutton 2018](https://mitpress.mit.edu/books/reinforcement-learning-second-edition) for more information. 

### Upper Confidence Bound

A limitation about $\epsilon$-greedy is that exploration is done indiscriminately. Is it possible to make a more informed choice about which alternative actions are more likely to yield a good result, so we preferentially pick one of them? That's what the Upper Confidence Bound (UCB) algorithm attempts to do. It weights some choices over others. 

It's worth looking at the formula that governs the choice for the next action at time $t$:

$$A_t \doteq \frac{argmax}{a}\bigg[ Q_t(a) + c\sqrt{ \dfrac{\ln(t)}{N_t(a)} }\bigg]$$

It's not essential to fully understand all the details, but here is the gist of it; the best action to take at time $t$, $A_t$, is decided by picking the best known action for returning the highest value (the $Q_t(a)$ term in the brackets [...] computes this), but with a correction that encourages exploration, especially for smaller $t$, but penalizing particular actions $a$ if we've already picked them a lot before (the second term starting with a constant $c$ that governs the "strength" of this correction).

UCB is one of the best performing algorithms [Sutton 2018](https://mitpress.mit.edu/books/reinforcement-learning-second-edition). How does it stack up against our properties:

1. Exploration is reasonably quick, governed by the $c$ hyperparameter for the "correction term".
2. It attempts to pick a good action when exploring, rather than randomly.
3. Finding the optimal action occurs efficiently, governed by the constant $c$.
4. The $ln(t)$ factor in the correction term grows more slowly over time relative to the counts $N_t(a)$, so exploration occurs less frequently.

#### UCB in More Detail

Let's explain the equation in more detail. If you are just interested in developing an intuition about strategies, this is a good place to stop and go to the [next lesson](03-Simple-Multi-Armed-Bandit.ipynb). 

* $A_t$ is the action we want to select at time $t$, the action that is most likely to produce the best reward or most likely to be worth exploring.
* For all the actions we can choose from, we pick the action $a$ that maximizes the formula in the brackets [...].
* $Q_t(a)$ is any equation we're using to measure the "value" received at time $t$ for action $a$. This is the greedy choice, i.e., the equation that tells us which action $a$ we currently know will give us the highest value. If we never wanted to explore, the second term in the brackets wouldn't exist. $Q_t(a)$ alone would always tell us to pick the best action we already know about. 
* The second term in the brackets is the correction that UCB gives us. As time $t$ increases, the natural log of $t$ also increases, but slower and slower for larger $t$. This is good because we hope we will find the real best action at some earlier time $t$, so exploration at large $t$ is less useful. However, the denominator, $N_t(a)$ is the number of times we've selected $a$ already. The more times we've already tried, the less "interesting" it is to try again, so this term penalizes choosing $a$. Finally, $c$ is a constant, a "knob" or _hyperparameter_ that determines how much we weight exploration vs. exploitation.


When we use UCB shortly, we'll use a simple _linear_ equation for $Q_t(a)$, i.e., something of the form $z = ax + by + c$.

See [Wikipedia - MAB Approximate solutions for contextual bandit](https://en.wikipedia.org/wiki/Multi-armed_bandit), [these references](../06-RL-References.ipynb#Upper-Confidence-Bound), and the [RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-upper-confidence-bound-contrib-linucb) for more information. 

### Thompson Sampling

Thompson sampling, developed in the 30s, is similar to UCB in that it picks the action that is believed to have the highest potential of maximum reward. The probability of selecting any of the available actions is based on the expectation of reward from the posterior distributions of rewards with previously sampled actions with contexts. In practice this can be computationally intractable for complex contexts, like deep learning models, but approximate Bayesian methods have made this more tractable ([paper](https://arxiv.org/abs/1802.09127)).

Thompson Sampling and UCB have similar performance characteristics and similar support for the desired properties we listed above.

For more information, see [Wikipedia](https://en.wikipedia.org/wiki/Thompson_sampling), [A Tutorial on Thompson Sampling](https://web.stanford.edu/~bvr/pubs/TS_Tutorial.pdf), [RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-thompson-sampling-contrib-lints), and other references in [06 RL References](../06-RL-References.ipynb).

### Gradient Bandit Algorithms

Focusing explicitly on rewards isn't the only approach. What if we use a more general measure, a _preference_, for selecting an action $a$ at time $t$, represented here $H_t(a)$? We need to model this so we have a probability of selecting an action $a$. Using the _soft-max distribution_ works, also known as the Gibbs or Boltzmann distribution:

$Pr\{A_t = a\} \doteq \frac{e^{H_t(a)}}{\sum^{k}_{b=1}e^{H_t(b)}} \doteq \pi_t(a)$

$\pi_t(a)$ is defined to encapsulate this formula for the probability of taking action $a$ at time $t$.

The term _gradient_ is used for this algorithm because the training update formula for $H_t(a)$ is very similar to the _stochastic gradient descent_ formula used in other ML problems. 

After an action $a$ is selected at a time $t$ and reward $R_t$ is received, the action preferences are updated as follows:

$ H_{t+1}(A_t) \doteq H_t(A_t) + \alpha(R_t - \overset{\_}{R_t})(1 - \pi_t(A_t))$,  and

$ H_{t+1}(a) \doteq H_t(a) - \alpha(R_t - \overset{\_}{R_t})(\pi_t(a))$,  for all $a \ne A_t$

where $H_0(a)$ values are initialized to zero, $\alpha > 0$ is a step size parameter and $\overset{\_}{R_t}$ is the average of all the rewards up through and including time $t$. Note that if $R_t - \overset{\_}{R_t}$ is positive, meaning the current reward is larger than the average, the preference $H(A_t)$ increases. Otherwise, it descreases. Note the plus vs. minus signs in the two equations before the $\alpha$ term. We would expect the _other_ $H(a)$ terms, for the rest of the $a$ actions, to decrease when $H(A_t)$ increases. That is, if our preference for $A_t$ increases, our preferences for the other actions should decrease.

How does it stack up against our properties:

1. As shown, this algorithm doesn't have tuning parameters to control the rate of exploration or convergence to the optimal solution. However, the convergence is reasonably quick if the variance in reward values is relatively high, so that the difference $R_t - \overset{\_}{R_t}$ is also relatively large for low $t$ values.
2. It attempts to pick a good action when exploring, rather than randomly.
3. See 1.
4. As $\overset{\_}{R_t}$ converges to a maximum, the difference $R_t - \overset{\_}{R_t}$ and hence all the preference values $H_t(a)$ will become relatively stationary, with the optimal action having the highest $H$. Since the $H_t(a)$ values govern the probability of being selected, based on the _soft-max distribution_, if the optimal action has a significantly higher $H_t(a)$ than the other actions, it will be chosen most frequently. If the differences between $H_t(a)$ values are not large, then several will be chosen frequently, but that also means their rewards are relatively close. Hence, in either case, the average reward over time will still be close to optimal.

We won't discuss this algorithm further. See [Sutton 2018](https://mitpress.mit.edu/books/reinforcement-learning-second-edition) for more details.